In [ ]:
pip install boto3

In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError
import os


def aws_config(id , key):
  os.environ["AWS_ACCESS_KEY_ID"] = id
  os.environ["AWS_SECRET_ACCESS_KEY"] = key

def check_open_security_groups(ec2_client):
    try:
        response = ec2_client.describe_security_groups()
        return [group['GroupId'] for group in response['SecurityGroups'] if any(rule['CidrIp'] == '0.0.0.0/0' for rule in rule['IpRanges']) for rule in group['IpPermissions']]
    except ec2_client.exceptions.ClientError as e:
        print(f"Error: {e}")
        return []

def check_cloudtrail_enabled(cloudtrail_client):
    try:
        response = cloudtrail_client.describe_trails()
        return len(response['trailList']) > 0
    except cloudtrail_client.exceptions.ClientError as e:
        print(f"Error: {e}")
        return False

def check_s3_bucket_public_access(s3_client):
    try:
        response = s3_client.list_buckets()
        buckets = response['Buckets']
        return [bucket['Name'] for bucket in buckets if s3_client.get_bucket_acl(Bucket=bucket['Name'])['Grants']]
    except s3_client.exceptions.ClientError as e:
        print(f"Error: {e}")
        return []

def check_aws_config_enabled(config_client):
    try:
        response = config_client.describe_configuration_recorders()
        return len(response['ConfigurationRecorders']) > 0
    except config_client.exceptions.ClientError as e:
        print(f"Error: {e}")
        return False

def main():
    # Specify your AWS region
    region = 'us-west-2'
    print("Enter access id: ")
    ACCESS_ID=input()
    print("Enter access key: ")
    ACCESS_KEY=input()

    # Create AWS clients
    iam_client = boto3.client('iam', region_name=region)
    ec2_client = boto3.client('ec2', region_name=region)
    cloudtrail_client = boto3.client('cloudtrail', region_name=region)
    s3_client = boto3.client('s3', region_name=region)
    config_client = boto3.client('config', region_name=region)

    # Check security best practices
    aws_config(ACCESS_ID , ACCESS_KEY)
    open_security_groups = check_open_security_groups(ec2_client)
    cloudtrail_enabled = check_cloudtrail_enabled(cloudtrail_client)
    public_s3_buckets = check_s3_bucket_public_access(s3_client)
    aws_config_enabled = check_aws_config_enabled(config_client)

    # Display results
    print(f"Open security groups: {open_security_groups}")
    print(f"AWS CloudTrail enabled: {cloudtrail_enabled}")
    print(f"S3 buckets with public access: {public_s3_buckets}")
    print(f"AWS Config enabled: {aws_config_enabled}")

if __name__ == "__main__":
    main()
